In [29]:
from herbie import Herbie
from pathlib import Path
import os
from datetime import date, timedelta

import numpy as np

import xarray as xr
#from netCDF4 import Dataset,num2date,date2num

In [24]:
PATH_ORIGINAL = f"/data1/projects/RTMA/alex.schein/Regridded_HRRR/d2m"

In [2]:
def restrict_files(START_DATE, END_DATE, TIME_LIST, PATH_ORIGINAL, PATH_NEW, IDX_MIN_LON=596, IDX_MIN_LAT=645, IMG_SIZE_LON=180, IMG_SIZE_LAT=180):
    # NOTE: default idx mins are for HRRR - need to change if using this for URMA!
    NUM_DAYS = END_DATE-START_DATE
    for i in range(NUM_DAYS.days+1):
        DATE_STR = date.strftime(START_DATE + timedelta(days=i), "%Y%m%d")
        filenames = os.listdir(f"{PATH_ORIGINAL}/{DATE_STR}")
        for time in TIME_LIST:
            filename = [x for x in filenames if f"t{time}z" in x and ".idx" not in x][0]
            new_filename = f"hrrr_regridded_spatiallyrestricted_{DATE_STR}_t{time}z.nc"
            if not os.path.exists(f"{PATH_NEW}/{new_filename}"):
                var = xr.open_dataset(f"{PATH_ORIGINAL}/{DATE_STR}/{filename}", engine="cfgrib", decode_timedelta=True)
                var_subset = var.isel(y=slice(IDX_MIN_LAT, IDX_MIN_LAT+IMG_SIZE_LAT),
                                      x=slice(IDX_MIN_LON, IDX_MIN_LON+IMG_SIZE_LON))
                var_subset.to_netcdf(f"{PATH_NEW}/{new_filename}")
                print(f"{new_filename} written to {PATH_NEW}")
            else:
                print(f"{new_filename} already exists in {PATH_NEW}. No action taken")
                
    return

In [66]:
def restrict_files_updated(START_DATE, END_DATE, FORECAST_LEAD_HOURS, PATH_ORIGINAL, PATH_NEW, IDX_MIN_LON=596, IDX_MIN_LAT=645, IMG_SIZE_LON=180, IMG_SIZE_LAT=180):
    NUM_HOURS = int((END_DATE-START_DATE).total_seconds()/3600) #surprisingly no built in hours function...
    for i in range(NUM_HOURS+1):
        DATE_STR = date.strftime(START_DATE + timedelta(hours=i), "%Y%m%d")
        filename_to_open = f"hrrr_regridded_{DATE_STR}_t{str((START_DATE.hour+i)%24).zfill(2)}z_f{str(FORECAST_LEAD_HOURS).zfill(2)}.grib2"
        new_filename = f"hrrr_regridded_spatiallyrestricted_{DATE_STR}_t{str((START_DATE.hour+i)%24).zfill(2)}z.nc"
        if not os.path.exists(f"{PATH_NEW}/{new_filename}"):
            var = xr.open_dataset(f"{PATH_ORIGINAL}/{DATE_STR}/{filename_to_open}", engine="cfgrib", decode_timedelta=True)
            var_subset = var.isel(y=slice(IDX_MIN_LAT, IDX_MIN_LAT+IMG_SIZE_LAT),
                                  x=slice(IDX_MIN_LON, IDX_MIN_LON+IMG_SIZE_LON))
            var_subset.to_netcdf(f"{PATH_NEW}/{new_filename}")
            print(f"{new_filename} written to {PATH_NEW}")
        else:
            print(f"{new_filename} already exists in {PATH_NEW}. No action taken")

In [68]:
#probably the easiest way is to adjust start and end dates to include hours, then adjust function to match

FORECAST_LEAD_HOURS = 1

START_DATE = dt.datetime(2021,1,1,0)-dt.timedelta(hours=FORECAST_LEAD_HOURS) 
END_DATE = dt.datetime(2023,12,31,23)-dt.timedelta(hours=FORECAST_LEAD_HOURS) 

START_DATE_TEST = dt.datetime(2024,1,1,0)-dt.timedelta(hours=FORECAST_LEAD_HOURS) 
END_DATE_TEST = dt.datetime(2024,12,31,23)-dt.timedelta(hours=FORECAST_LEAD_HOURS) 

# START_DATE_TRAIN = dt.datetime(2020,12,31,23) #should be 00 UTC jan 1, 2021 minus forecast time (e.g. f01 --> 2020/12/31 23z)
# END_DATE_TRAIN = dt.datetime(2023,12,31,22) #should be 23 UTC dec 31, 2023 (e.g. f01 --> 2023/12/31 22z)

# START_DATE_TEST = dt.datetime(2023,12,31,23) #should be 00 UTC jan 1, 2024 minus forecast time (e.g. f01 --> 2023/12/31 23z)
# END_DATE_TEST = dt.datetime(2024,12,31,22) #should be 23 UTC dec 31, 2024 minus forecast time (e.g. f01 --> 2024/12/31 22z)

In [69]:
(START_DATE.hour+25)%24

0

In [80]:
i=20000
DATE_STR = date.strftime(START_DATE + timedelta(hours=i), "%Y%m%d")
filename_to_open = f"hrrr_regridded_{DATE_STR}_t{str((START_DATE.hour+i)%24).zfill(2)}z_f{str(FORECAST_LEAD_HOURS).zfill(2)}.grib2"
new_filename = f"hrrr_regridded_spatiallyrestricted_{DATE_STR}_t{str((START_DATE.hour+i)%24).zfill(2)}z.nc"
new_filename

'hrrr_regridded_spatiallyrestricted_20230414_t07z.nc'

In [53]:
DATE_STR = date.strftime(START_DATE_TRAIN + timedelta(days=1), "%Y%m%d")
filenames = os.listdir(f"{PATH_ORIGINAL}/{DATE_STR}")
filenames = [file for file in filenames if ".idx" not in file]
times = [filename[25:27] for filename in filenames] #relies on naming scheme of "hrrr_regridded_YYYYMMDD_tHHz_..." which SHOULD always be the case

In [65]:
filenames

['hrrr_regridded_20210101_t00z_f01.grib2',
 'hrrr_regridded_20210101_t01z_f01.grib2',
 'hrrr_regridded_20210101_t02z_f01.grib2',
 'hrrr_regridded_20210101_t03z_f01.grib2',
 'hrrr_regridded_20210101_t04z_f01.grib2',
 'hrrr_regridded_20210101_t05z_f01.grib2',
 'hrrr_regridded_20210101_t06z_f01.grib2',
 'hrrr_regridded_20210101_t07z_f01.grib2',
 'hrrr_regridded_20210101_t08z_f01.grib2',
 'hrrr_regridded_20210101_t09z_f01.grib2',
 'hrrr_regridded_20210101_t10z_f01.grib2',
 'hrrr_regridded_20210101_t11z_f01.grib2',
 'hrrr_regridded_20210101_t12z_f01.grib2',
 'hrrr_regridded_20210101_t13z_f01.grib2',
 'hrrr_regridded_20210101_t14z_f01.grib2',
 'hrrr_regridded_20210101_t15z_f01.grib2',
 'hrrr_regridded_20210101_t16z_f01.grib2',
 'hrrr_regridded_20210101_t17z_f01.grib2',
 'hrrr_regridded_20210101_t18z_f01.grib2',
 'hrrr_regridded_20210101_t19z_f01.grib2',
 'hrrr_regridded_20210101_t20z_f01.grib2',
 'hrrr_regridded_20210101_t21z_f01.grib2',
 'hrrr_regridded_20210101_t22z_f01.grib2',
 'hrrr_regr